In [ ]:
import vk

In [ ]:
app_id = 
token = 

In [ ]:
vksession = vk.Session(access_token=token)
vkapi = vk.API(vksession)

Getting messages:

In [ ]:
messages_get_params = {
    'out': '0', # 1 — to return outgoing messages , 0 — to return incoming messages (default) 
    'offset': '0', # Offset needed to return a specific subset of messages. positive number
    'count': '200', # Number of messages to return. positive number, default 20, maximum value 200
    'filters': '0' # Filter to apply: 
                  # 1 — unread only 
                  # 2 — not from the chat 
                  # 4 — messages from friends 
                  # If the 4 flag is set, the 1 and 2 flags are not considered. 
                  # positive number, default 0
}

In [ ]:
msgs = vkapi.messages.get(**messages_get_params)

In [ ]:
msgs

In [ ]:
vkapi.messages.getLastActivity(user_id='47244995')

Message history with certain user, simple:

In [ ]:
messages_gethistory_params = {
    'offset': '0',
    'count': '200', # positive number, default 20, maximum value 200
    'user_id': '12135262',
    'rev': '1' # 1 — return messages in chronological order. 0 — return messages in reverse chronological order. 
}

In [ ]:
hist_msgs = vkapi.messages.getHistory(**messages_gethistory_params)

In [ ]:
hist_msgs

Message history, optimized query. vk.com rejects to send more than 4000 messages in single execute call.

In [ ]:
msg_query_part = """API.messages.getHistory({"offset": %d, "count": 200, "user_id": %d, "rev": %d})+"""
msg_query = "return "
for i in range(0,4000,200):
    msg_query += msg_query_part % (i, 
                                   int(messages_gethistory_params['user_id']), 
                                   int(messages_gethistory_params['rev']))
msg_query = msg_query.strip("+")
msg_query += ";"

In [ ]:
query_result = vkapi.execute(code=msg_query)
query_result = [x for x in query_result if type(x) is not int] # remove total counts sent with each query

In [ ]:
query_result

In [ ]:
print(vkapi.messages.getHistory(user_id="12135262",count=0)[0], # one of the ways of getting total count of messages
vkapi.messages.getHistory(chat_id="84",count=0)[0]) # one of the ways of getting total count of messages

In [ ]:
def get_all_messages_from(vkapi, user_id, is_chat=True): # if is_chat, user_id is treated as chat_id
    import time
    
    messages_gethistory_params = {
        'offset': '0',
        'count': '200',
        'user_id': str(user_id),
        'rev': '1'
    }
    
    all_messages = []
    current_bulk = ["placeholder"]
    n = 0
    
    total = (vkapi.messages.getHistory(chat_id=messages_gethistory_params["user_id"],count=0) 
             if is_chat 
             else vkapi.messages.getHistory(user_id=messages_gethistory_params["user_id"],count=0))[0]
    print("Going to fetch %d messages" % total)
    
    while True:
        msg_query_part = """API.messages.getHistory({"offset": %d, "count": 200, """ + (""""user_id" """ if not is_chat else """"chat_id" """) + """: %d, "rev": %d})+"""
        msg_query = "return "
        for i in range(n,n+4000,200):
            msg_query += msg_query_part % (i, 
                                           int(messages_gethistory_params['user_id']), 
                                           int(messages_gethistory_params['rev']))
        msg_query = msg_query.strip("+")
        msg_query += ";"
        
        current_bulk = vkapi.execute(code=msg_query)
        time.sleep(1)
        
        all_messages.append(current_bulk)
        n += len(current_bulk)
        if type(current_bulk[-1]) is int:
            break
        
        print("Inserted %d elements.\nlast: %s" % (n,all_messages[-1][-1]))
        
    flat = [y for x in all_messages for y in x]
    cleaned = [x for x in flat if type(x) is not int]  
    return cleaned

In [ ]:
#m = get_all_messages_from(vkapi,47244995)
m = get_all_messages_from(vkapi,84)

Adding 'user' field with actual usernames and vk page links

In [ ]:
from vk_helper import get_user
user_set = set(dataitem['uid'] for dataitem in m)
user_data = [get_user(vkapi, u)[0] for u in user_set]
for d in m:
    d['user'] = [u for u in user_data if u['uid'] == d['uid']][0]

Save messages' dump to file

In [ ]:
import json
with open('chat_kotanat.json', 'w') as f:
    json.dump(m, f)

In [ ]:
with open('chat_kotanat.json','r') as f:
    data = json.load(f)
len(data)

In [ ]:
users = construct_user_set_from_data(vkapi,data)

In [ ]:
for user in users:
    if len(user) == 3:
        print(user[0],user[1],"(%s)" % user[2])
    if len(user) == 4:
        print(user[0],user[1],"(%s)" % user[2]," - " + user[3])